In [1]:
from cobra.io import read_sbml_model
from cobra import Reaction, Metabolite

# Read model (model of Escherichia coli)
model = read_sbml_model('iML1515.xml')

In [2]:
model.metabolites.get_by_id('34hpp_c')
#starting point
#add 5 reactions: 
#homogentisate, 
#2-Methyl-6-phytylquinol, 
#2,3-Dimethyl-5-phytylquinol, 
#gamma-Tocopherol,
#alpha-Tocopherol

Metabolite identifier,34hpp_c
Name,3-(4-Hydroxyphenyl)pyruvate
Memory address,0x7ff4a1f2f6d0
Formula,C9H7O4
Compartment,c
In 2 reaction(s),"TYRTA, PPND"


In [4]:
# Reaction 1: 3-(4-Hydroxyphenyl)pyruvate + O2 O2 --> CO2 CO2 + Homogentisate
new_reaction1 = Reaction('34HPPOR') # 4 Hydroxyphenylpyruvateoxygen oxidoreductase
homogentisate = Metabolite(id='hgentis_c', compartment='c')
new_reaction1.add_metabolites({model.metabolites.get_by_id('34hpp_c'): -1, # 3-(4-Hydroxyphenyl)pyruvate
                               model.metabolites.o2_c: -1, #O2
                               homogentisate: 1, # Homogentisate
                               model.metabolites.co2_c: 1, # CO2
                              })
model.add_reactions([new_reaction1])

In [5]:
# Reaction 2: h_c + hgentis_c + phdp_c ⇌ co2_c + ppi_c + 2m6phol_c
new_reaction2 = Reaction('HGPHT') # Homogenitisate phytyltransferase
methylphytylquinol = Metabolite(id='2m6phol_c', compartment='c')
phytyldiphosphate = Metabolite(id='phdp_c', compartment='c') # Phytyl diphosphate
new_reaction2.add_metabolites({model.metabolites.hgentis_c: -1, # Homogentisate
                               model.metabolites.h_c: -1, # H+
                               phytyldiphosphate: -1,
                               methylphytylquinol: 1, # 2-Methyl-6-phytylquinol
                               model.metabolites.ppi_c: 1, # diphosphate
                               model.metabolites.co2_c: 1, # CO2
                              })
model.add_reactions([new_reaction2])

In [7]:
# Reaction 3: amet_c + 2m6phol_c ⇌ ahcys_c + h_c + 23dmphol_c
new_reaction3 = Reaction('MPBQ') # 2-methyl-6-phytylbenzoquinone methyltransferase
dimethylphytylquinol = Metabolite(id='23dmphol_c', compartment='c')
new_reaction3.add_metabolites({model.metabolites.get_by_id('2m6phol_c'): -1, # 2-Methyl-6-phytylquinol
                               model.metabolites.amet_c: -1, # S-Adenosyl-L-methionine
                               dimethylphytylquinol: 1, # 2,3-Dimethyl-5-phytylquinol
                               model.metabolites.ahcys_c: 1, # S-Adenosyl-L-homocysteine
                               model.metabolites.h_c: 1, # H+
                              })
model.add_reactions([new_reaction3])

In [8]:
# Reaction 4: 23dmphol_c ⇌ gtocophe_c
new_reaction4 = Reaction('TOCOPHS1') # Tocopherol cyclase (g)
gammatocopherol = Metabolite(id='gtocophe_c', compartment='c')
new_reaction4.add_metabolites({model.metabolites.get_by_id('23dmphol_c'): -1, # 2,3-Dimethyl-5-phytylquinol
                               gammatocopherol: 1, # Gamma-Tocopherol
                              })
model.add_reactions([new_reaction4])

In [9]:
# Reaction 5: amet_c + gtocophe_c ⇌ ahcys_c + h_c + avite1_c
new_reaction5 = Reaction('TOCOPHOM1') # Tocopherol O-methyltransferase
alphatocopherol = Metabolite(id='avite1_c', compartment='c')
new_reaction5.add_metabolites({model.metabolites.gtocophe_c: -1, # Gamma-Tocopherol
                               model.metabolites.amet_c: -1, # S-Adenosyl-L-methionine
                               alphatocopherol: 1, # Alpha-Tocopherol
                               model.metabolites.ahcys_c: 1, # S-Adenosyl-L-homocysteine
                               model.metabolites.h_c: 1, # H+
                              })
model.add_reactions([new_reaction5])

In [18]:
# Co-factor production
# Reaction 6: frdp_c + ipdp_c ⇌ ggdp_c + h_c + ppi_c
new_reaction6 = Reaction('GGPS') # Geranylgeranyl diphosphate synthase
geranylgeranyldiphosphate = Metabolite(id='ggdp_c', compartment='c')
new_reaction6.add_metabolites({model.metabolites.frdp_c: -1, # Farnesyl diphosphate
                               model.metabolites.ipdp_c: -1, # Isopentenyl diphosphat
                               geranylgeranyldiphosphate: 1, # Geranylgeranyl diphosphate
                               model.metabolites.ppi_c: 1, # diphosphate
                               model.metabolites.h_c: 1, # H+
                              })
model.add_reactions([new_reaction6])

In [22]:
# Co-factor production
# Reaction 7: ggdp_c + 3.0 h_c + 3.0 nadph_c ⇌ 3.0 nadp_c + phdp_c
new_reaction7 = Reaction('GGDR') # Geranylgeranyl diphosphate reductase
new_reaction7.add_metabolites({model.metabolites.ggdp_c: -1, # Geranylgeranyl diphosphate
                               model.metabolites.h_c: -3, # H+
                               model.metabolites.nadph_c: -3, # NADPH
                               model.metabolites.nadp_c: 3, # NADP
                               model.metabolites.phdp_c: 1, # # Phytyl diphosphate
                              })
model.add_reactions([new_reaction7])

In [23]:
#production of vitamin E (avite1_c)
model.add_boundary(model.metabolites.avite1_c, type='demand')

with model:
    model.objective = model.reactions.TOCOPHOM1
    lets_produce_avite1 = model.optimize().objective_value

In [24]:
#production rate of vitamin E
lets_produce_avite1

1.222850031113874